In [1]:
import pandas as pd
import numpy as np
import os  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

In [2]:
df_demanda = pd.read_csv("../../dados_preparados/dados_tratamento_inicial.csv")

In [3]:
df_demanda.head()

,Unnamed: 0,id_demanda,frequencia_uso,objetivo,score,situacao_atual,titulo_demanda,tipo_solicitante,cargo_solicitante,departamento_solicitante,nome_solicitante,setor_solicitante,qtd_beneficio_real,qtd_beneficio_potencial,qtd_beneficio_qualitativo,valor_total_beneficios_reais (BRL),valor_total_beneficios_potenciais (BRL),descricao_beneficios,quantidade_CC,nomes_CC
0,0,1,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 01,Solicitante,Técnico,TI,Diego,Suporte Técnico,0,0,2,0.0,0.0,. Sit amet porttitor eget dolor morbi non arcu...,1,. Departamento de Engenharia de Sistemas
1,1,2,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 02,Solicitante,Técnico,TI,Diego,Suporte Técnico,0,0,2,0.0,0.0,. Sit amet porttitor eget dolor morbi non arcu...,1,. Departamento de Engenharia de Sistemas
2,2,3,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 03,Solicitante,Técnico,TI,Henrique,Administração de Redes e Sistemas Internos,0,1,1,0.0,78.0,. Sit amet porttitor eget dolor morbi non arcu...,1,. Departamento de Segurança Digital
3,3,4,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 04,Solicitante,Técnico,TI,Henrique,Administração de Redes e Sistemas Internos,0,1,1,0.0,78.0,. Sit amet porttitor eget dolor morbi non arcu...,1,. Departamento de Segurança Digital
4,4,11,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",12.0,Duis aute irure dolor in reprehenderit in volu...,Demanda Teste Histórico 01,Solicitante,Analista de Dados,TI,Eduarda,Segurança Dital,0,0,0,0.0,0.0,NaN,1,. Departamento de Desenvolvimento de Software


In [4]:
df_demanda.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df_demanda.columns

Index(['id_demanda', 'frequencia_uso', 'objetivo', 'score', 'situacao_atual',
       'titulo_demanda', 'tipo_solicitante', 'cargo_solicitante',
       'departamento_solicitante', 'nome_solicitante', 'setor_solicitante',
       'qtd_beneficio_real', 'qtd_beneficio_potencial',
       'qtd_beneficio_qualitativo', 'valor_total_beneficios_reais (BRL)',
       'valor_total_beneficios_potenciais (BRL)', 'descricao_beneficios',
       'quantidade_CC', 'nomes_CC'],
      dtype='object')

Colunas para o tratamento de NLP 

* frequencia_uso                 
* objetivo
* situacao_atual
* titulo_demanda
* tipo_solicitante
* cargo_solicitante
* departamento_solicitante
* nome_solicitante
* descricao_beneficios
* nomes_CC

Colunas para a parte não NLP 

* score                 
* qtd_beneficio_real
* qtd_beneficio_potencial
* qtd_beneficio_qualitativo
* valor_total_beneficios_reais (BRL)
* valor_total_beneficios_potenciais (BRL)
* quantidade_CC

In [6]:
def adicionarNovaColuna(df_alvo, nome_coluna, df_info=df_demanda):
    X = df_info[nome_coluna].values.astype('U')

    cvec = CountVectorizer().fit(X)
    df_coluna = pd.DataFrame(cvec.transform(X).todense(), columns=cvec.get_feature_names_out())
    
    return df_coluna

In [8]:
df_preparacao_NLP = pd.DataFrame()
nome_colunas_desejadas = [
    "frequencia_uso", "objetivo", "situacao_atual","titulo_demanda","tipo_solicitante","cargo_solicitante",
    "departamento_solicitante","nome_solicitante","descricao_beneficios","nomes_CC"
]

for nome_coluna in df_demanda.columns:
    if nome_coluna in nome_colunas_desejadas:
        df_coluna_nova = adicionarNovaColuna(df_preparacao_NLP, nome_coluna)
        df_preparacao_NLP = pd.concat([df_preparacao_NLP, df_coluna_nova], axis=1)
        
        
df_preparacao_NLP.head()

,diariamente,mensalmente,semanalmente,11,ad,adipiscing,aliqua,aliquip,amet,atualizar,...,risus,sit,de,departamento,desenvolvimento,digital,engenharia,segurança,sistemas,software
0,0,0,1,0,1,1,1,1,1,0,...,2,2,2,1,0,0,1,0,1,0
1,0,0,1,0,1,1,1,1,1,0,...,2,2,2,1,0,0,1,0,1,0
2,0,0,1,0,1,1,1,1,1,0,...,2,2,1,1,0,1,0,1,0,0
3,0,0,1,0,1,1,1,1,1,0,...,2,2,1,1,0,1,0,1,0,0
4,0,0,1,0,1,1,1,1,1,0,...,0,0,2,1,1,0,0,0,0,1
